# C3S examples 
## Retrieve and visualize GRIB files

## 1 - Configuration

### 1.1 - SET YOUR C3S ACCOUNT (FOR THE AUTHENTIFICATION) 

In [ ]:
# For Registration (C3S)
url_registration= 'https://cds.climate.copernicus.eu/user/register?destination=%2F%23!%2Fhome'
from IPython.core.display import display, HTML
display(HTML(f"""<a href={url_registration}>{'Registration_C3S - please click on it'}</a>"""))

### 1.2 - Log in and retrieve your key

In [ ]:
url_login= 'https://cds.climate.copernicus.eu/user/login?destination=%2F%23!%2Fhome'
url_key = 'https://cds.climate.copernicus.eu/api-how-to'

display(HTML(f"""<a href={url_login}>{'Please login - please click on it'}</a>"""))
display(HTML(f"""<a href={url_key}>{'Retrieve your key - please click on it'}</a>"""))

### 1.3- Install the CDS API key

In [ ]:
import os

print('Please enter your key:')
key = input()

content = "url: https://cds.climate.copernicus.eu/api/v2\nkey: " + key

#Write content in $HOME/.cdsapirc 
f = open(os.environ["HOME"] + '/cdsapirc', 'w')
f.write(content)
f.close()
!mv $HOME/cdsapirc $HOME/.cdsapirc

## 2 - Consult the catalogue

#### Choose a dataset, click on "Download data", select the desired parameters, agree to the Terms of Use and finally, click on "Show API request" to see the Python script

In [ ]:
url_dataset= 'https://cds.climate.copernicus.eu/cdsapp#!/search?type=dataset'
display(HTML(f"""<a href={url_dataset}>{'Consult the catalogue - please click on it'}</a>"""))

## 3 - Reanalysis

### 3.1 - Create a data folder

In [ ]:
#Get the destination folder path
folder = '/home/jovyan/c3s_data/c3s_grib_examples/03_reanalysis'

import os 

#Create the folder if it doesn't exist
if not os.path.exists(folder):
    os.makedirs(folder)

filename = folder + '/temperature.grib' #Give a name to your file

### 3.2 - Download a file
#### TO DO : Check you agree with the __[Terms of Use](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels-monthly-means?tab=form)__ (Last section)
#### Note : Please, go to https://cds.climate.copernicus.eu/cdsapp#!/yourrequests to track the progress of your request

In [ ]:
import cdsapi

years = ['1988', '1998', '2008', '2018']

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-single-levels-monthly-means',
    {
        'format':'grib',
        'month':[
            "01",
            "02",
            "03",
            "04",
            "05",
            "06",
            "07",
            "08",
            "09",
            "10",
            "11",
            "12"
          ],
        'product_type':'monthly_averaged_reanalysis',
        'variable':'2m_temperature',
        'year':years,
        'time':'00:00'
    },
    filename)

### 3.3 - Display a map

#### Note: The file must be in Grib format

In [ ]:
os.environ["PROJ_LIB"] = "/opt/conda/share/proj";

from IPython.display import Image
import pygrib 
import numpy as np
import ipywidgets as widgets

from matplotlib.pyplot import figure
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from datetime import datetime
import imageio

# Open a GRIB file
grbs = pygrib.open(filename)

grb = grbs.select()[0]
vmax = np.amax(grb.values)
vmin = np.amin(grb.values)

# Get the longitudes and the latitudes 
latitudes,longitudes = grb.latlons() 

unit = grb['units']

progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=len(grbs.select()),
    step=1,
    description='Loading:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)

display(progress_bar)

images = []

# Create and save a figure for each grib message
for grb in grbs.select():
    figure(num=None, figsize=(16, 10))
    
    date = datetime(year=grb['year'], month=grb['month'], day=grb['day'], \
                hour=grb['hour'], minute=grb['minute'], second=grb['second'])

    # Create a map
    map = Basemap(projection='cyl', lat_ts=10,llcrnrlon=longitudes.min(), \
     urcrnrlon=longitudes.max(),llcrnrlat=latitudes.min(),urcrnrlat=latitudes.max(), \
     resolution='l')

    map.drawcoastlines()

    x,y = map(longitudes, latitudes)
    data = grb.values

    #Colour the map 
    map.pcolormesh(x, y, data, cmap=plt.cm.coolwarm, vmin=vmin, vmax=vmax)
    map.colorbar(label=unit)

    plt.title(grb['parameterName'] + ' date: ' + date.strftime("%Y-%m-%d %H:%M:%S") + \
              ' step: ' + str(grb['stepRange']) + ' level:' + str(grb['level']))
    
    
    plt.savefig(os.path.join(folder, 'result_' + str(grb.messagenumber)))
    plt.close()
    
    images.append(imageio.imread(os.path.join(folder, 'result_' + str(grb.messagenumber)+ '.png')))
    
    progress_bar.value += 1

result_filename = "result.gif"

#Create a GIF
imageio.mimsave(os.path.join(folder, result_filename), images, 'GIF', duration=0.3)

display(Image(os.path.join(folder, result_filename)))

### 3.5 - Create a chart

In [ ]:
import calendar
import pandas as pd 

df = pd.DataFrame(index = [calendar.month_abbr[i] for i in range(1,13)])

i = 0 
#For each year
for year in years:
    means = []
    #For each month, add the mean in a list
    for grb in grbs.select(year=[int(year), int(year)-1]):
        means.append(np.mean(grb.values))
    df.insert(i, year, means, True) 
    i += 1

ax = df.plot.bar(rot=0, figsize=(12, 6))
ax.set_ylim(np.amin(means), np.amax(means))

plt.title(grbs.select()[0]['name'])
plt.ylabel(grbs.select()[0]['units'])
plt.legend()
plt.show()

# # Uncomment this section to see another chart
# import calendar
# for year in years:
#     means = []
#     for grb in grbs.select(year=[int(year), int(year)-1]):
#         means.append(np.mean(grb.values))
#     plt.plot([calendar.month_abbr[i] for i in range(1,13)] , means, label=year)

# plt.title(grbs.select()[0]['name'])
# plt.ylabel(grbs.select()[0]['units'])
# plt.legend()
# plt.show()

## 4 - Seasonal forecasts

### 4.1 - Create a data folder

In [ ]:
#Get the destination folder path
folder = '/home/jovyan/work/c3s_data/c3s_grib_examples/04_Seasonal_forecasts'

import os 

#Create the folder if it doesn't exist
if not os.path.exists(folder):
    os.makedirs(folder)

filename = folder + '/precipitation_anomalous_rate.grib' #Give a name to your file

### 4.2 - Download a file
#### TO DO : Check you agree to the  __[Terms of Use](https://cds.climate.copernicus.eu/cdsapp#!/dataset/seasonal-original-single-levels?tab=form)__ (Last section)
#### Note : Please, go to https://cds.climate.copernicus.eu/cdsapp#!/yourrequests to track the progress of your request

In [ ]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'seasonal-postprocessed-single-levels',
    {
        'format':'grib',
        'originating_centre':'ecmwf',
        'system':'5',
        'variable':'total_precipitation_anomalous_rate_of_accumulation',
        'product_type':'ensemble_mean',
        'year':[
            '2017','2018'
        ],
        'month':[
            '01','02','03',
            '04','05','06',
            '07','08','09',
            '10','11','12'
        ],
        'leadtime_month':'1'
    },
    filename)

#### 4.3 - Display a map

In [ ]:
os.environ["PROJ_LIB"] = "/opt/conda/share/proj";

from IPython.display import Image
import pygrib 
import numpy as np
import ipywidgets as widgets

from matplotlib.pyplot import figure
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from datetime import datetime
import imageio

# Open a GRIB file
grbs = pygrib.open(filename)

grb = grbs.select()[0]
vmax = np.amax(grb.values)
vmin = np.amin(grb.values)

# Get the longitudes and the latitudes 
latitudes,longitudes = grb.latlons() 

unit = grb['units']

progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=len(grbs.select()),
    step=1,
    description='Loading:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)

display(progress_bar)

images = []

# Create and save a figure for each grib message
for grb in grbs.select():
    figure(num=None, figsize=(16, 10))
    
    date = datetime(year=grb['year'], month=grb['month'], day=grb['day'], \
                hour=grb['hour'], minute=grb['minute'], second=grb['second'])

    # Create a map
    map = Basemap(projection='cyl', lat_ts=10,llcrnrlon=longitudes.min(), \
     urcrnrlon=longitudes.max(),llcrnrlat=latitudes.min(),urcrnrlat=latitudes.max(), \
     resolution='l')

    map.drawcoastlines()

    x,y = map(longitudes, latitudes)
    data = grb.values

    #Colour the map 
    map.pcolormesh(x, y, data, cmap=plt.cm.jet, vmin=vmin, vmax=vmax)
    map.colorbar(label=unit)

    plt.title(grb['parameterName'] + ' date: ' + date.strftime("%Y-%m-%d %H:%M:%S") + \
              ' step: ' + str(grb['stepRange']) + ' level:' + str(grb['level']))
    
    
    plt.savefig(os.path.join(folder, 'result_' + str(grb.messagenumber)))
    plt.close()
    
    images.append(imageio.imread(os.path.join(folder, 'result_' + str(grb.messagenumber)+ '.png')))
    
    progress_bar.value += 1

result_filename = "result.gif"

#Create a GIF
# images = [ imageio.imread(os.path.join(folder, 'result_' + str(i)+ '.png')) \
#           for i in range(1, grbs.messages + 1)]
imageio.mimsave(os.path.join(folder, result_filename), images, 'GIF', duration=0.3)

display(Image(os.path.join(folder, result_filename)))

In [ ]:
import calendar
from matplotlib.pyplot import figure

figure(num=None, figsize=(10, 6))

years = [2017, 2018]

for y in years:
    means = []
    
    gribs = grbs.select(year=y)
    for grb in gribs :
        means.append(np.mean(grb.values))
    plt.plot([calendar.month_abbr[i] for i in range(1,13)], means, label=str(y))

plt.title(grbs.select()[0]['name'])
plt.ylabel(grbs.select()[0]['units'])
plt.legend()    
plt.show()